In [3]:
import conceptnet_lite
conceptnet_lite.connect("../OPENAI/data/conceptnet.db")
from conceptnet_lite import Label, edges_for, edges_between

In [4]:

def bag_of_synonyms(word):
    sinonimos=set()
    try:
        for e in edges_for(Label.get(text=word, language='en').concepts, same_language=True):
            if e.relation.name == "synonym":
                if word== e.start.text:
                    sinonimos.add(e.end.text)
                elif word== e.end.text:
                    sinonimos.add(e.start.text)
    except:
        pass
    sinonimos.add(word)
    return sinonimos

def bag_of_antonyms(word):
    antonimos=set()
    try:
        for e in edges_for(Label.get(text=word, language='en').concepts, same_language=True):
            if e.relation.name in ["antonym","distinc_from"]:
                if word== e.start.text:
                    antonimos.add(e.end.text)
                elif word== e.end.text:
                    antonimos.add(e.start.text)
    except:
        pass
    return antonimos

relaciones=["is_a","etymologically_related_to","manner_of","has_a","derived_from","has_property","form_of","causes","has_prerequisite","has_subevent","has_first_subevent"]

def bag_of_hyperonyms(word):
    hiperonimos=set()
    try:
        for e in edges_for(Label.get(text=word, language='en').concepts, same_language=True):
            if e.relation.name in relaciones:
                if word== e.start.text:
                    hiperonimos.add(e.end.text)
    except:
        pass
    return hiperonimos

def bag_of_hyponyms(word):
    hiponimos=set()
    try:
        for e in edges_for(Label.get(text=word, language='en').concepts, same_language=True):
            if e.relation.name in relaciones:
                if word== e.end.text:
                    hiponimos.add(e.start.text)
    except:
        pass
    return hiponimos


In [5]:
def jaro_distance(s1, s2) :
    sinT=[]
    sinH=[]
    HipT=[]
    hipH=[]

    for t in s1:
        sinT.append(bag_of_synonyms(t))
        #HipT.append(bag_of_hyperonyms(t))
    for h in s2:
        sinH.append(bag_of_synonyms(h))
        #hipH.append(bag_of_hyponyms(h))
    
    tempSet=set()
    for i in range(len(sinT)):
        tempSet=set()
        for s_ in sinT[i]:
            tempSet=tempSet.union(bag_of_hyperonyms(s_))
        HipT.append(tempSet)
    for i in range(len(sinH)):
        tempSet=set()
        for s_ in sinH[i]:
            tempSet=tempSet.union(bag_of_hyponyms(s_))
        hipH.append(tempSet)
            
        
    print("sinonimos de T",sinT)
    print("Hiperonimos de T",HipT)
    print("sinonimos de H",sinH)
    print("hiponimos de h",hipH)

    bandera=True

    # Length of two strings
    len1 = len(s1)
    len2 = len(s2)

    # If the listas de tokens are equal 
    if len1==len2:
        for i in range(len1):
            if s1[i]!=s2[i]:
                bandera=False
                break
        if (bandera):
            return 1.0,0.0; 
 
    if (len1 == 0 or len2 == 0) :
        return 0.0,0.0; 
 
    # Maximum distance upto which matching 
    # is allowed 
    max_dist = (max(len(s1), len(s2)) // 2 )-1 ; 
 
    # Count of matches 
    match = 0; 
 
    # Hash for matches 
    hash_s1 = [0] * len(s1)
    hash_s2 = [0] * len(s2)
 
    # Traverse through the first string 
    for i in range(len1) : 
            
        # Check if there is any matches 
        for j in range( max(0, i - max_dist),
                    min(len2, i + max_dist + 1)) : 
            print(s1[i],s2[j])
            # If there is a match or is contain in a bag of sinomys of tk
            if ((s1[i] == s2[j] or s1[i] in sinH[j] or s2[j] in sinT[i]) and hash_s2[j] == 0) : 
                print(s1[i],s2[j],"sinonimos")
                hash_s1[i] += 1; 
                hash_s2[j] += 1; 
                match += 1; 
                break
            elif ((s1[i] in hipH[j] or len((sinT[i]).intersection(hipH[j]))>0) and hash_s2[j] == 0):
                print("hiperonimos",s2[j],s1[i],(sinT[i]).intersection(hipH[j]))
                hash_s1[i] += 1; 
                hash_s2[j] += 1; 
                match += 1; 
                break
            elif ((s2[j] in HipT[i] or len((sinH[j]).intersection(HipT[i]))>0) and hash_s2[j] == 0): 
                print("hiperonimos sobre sinonimos",s2[j],s1[i])
                hash_s1[i] += 1; 
                hash_s2[j] += 1; 
                match += 1; 
                break
            elif len((hipH[j]).intersection(HipT[i]))>0 and hash_s2[j] == 0: 
                print("hiperonimos3",s2[j],s1[i],(hipH[j]).intersection(HipT[i]))
                hash_s1[i] += 1; 
                hash_s2[j] += 1; 
                match += 1; 
                break
    print(hash_s1)
    print(hash_s2)
    print(match)
    # If there is no match 
    if (match == 0) :
        return 0.0,0.0; 
 
    # Number of transpositions 
    t = 0; 
 
    point = 0; 
 
    # Count number of occurrences 
    # where two characters match but 
    # there is a third matched character 
    # in between the indices 
    for i in range(len1) : 
        if (hash_s1[i]) :
 
            # Find the next matched character 
            # in second string 
            while (hash_s2[point] == 0) :
                point += 1; 
 
            if (s1[i] != s2[point]) :
                point += 1
                t += 1
            else :
                point += 1
                 
    t /= 2; 
    print(t)
    #Return the Jaro Similarity 
    return ((match / len2 +
            (match - t) / match ) / 2.0),t; 


In [6]:
def jaro_distance_relacionadas(s1, s2) :
    sinT=[]
    sinH=[]
    HipT=[]
    hipH=[]

    for t in s1:
        sinT.append(bag_of_synonyms(t))
        #hipT.append(bag_of_hyperonyms(t))
    for h in s2:
        sinH.append(bag_of_synonyms(h))
        #HipH.append(bag_of_hyponyms(h))
        
        
    tempSet=set()
    for i in range(len(sinT)):
        tempSet=set()
        for s_ in sinT[i]:
            tempSet=tempSet.union(bag_of_hyperonyms(s_))
        HipT.append(tempSet)
    for i in range(len(sinH)):
        tempSet=set()
        for s_ in sinH[i]:
            tempSet=tempSet.union(bag_of_hyponyms(s_))
        hipH.append(tempSet)
        
        
    print("sinonimos de T",sinT)
    print("Hiperonimos de T",HipT)
    print("sinonimos de H",sinH)
    print("hiponimos de h",hipH)

    bandera=True

    # Length of two strings
    len1 = len(s1)
    len2 = len(s2)

    # If the listas de tokens are equal 
    if len1==len2:
        for i in range(len1):
            if s1[i]!=s2[i]:
                bandera=False
                break
        if (bandera):
            return 1.0,0.0; 
 
    if (len1 == 0 or len2 == 0) :
        return 0.0,0.0; 
 
    # Maximum distance upto which matching 
    # is allowed 
    max_dist = (max(len(s1), len(s2)) // 2 ) -1; 
 
    # Count of matches 
    match = 0; 
 
    # Hash for matches 
    hash_s1 = [0] * len(s1)
    hash_s2 = [0] * len(s2)
 
    # Traverse through the first string 
    for i in range(len1) : 
            
        # Check if there is any matches 
        for j in range( max(0, i - max_dist),
                    min(len2, i + max_dist + 1)) : 
            #print(s1[i],s2[j])
            # If there is a match or is contain in a bag of sinomys of tk
            if ((s1[i] in hipH[j] or len((sinT[i]).intersection(hipH[j]))>0) and s1[i]!=s2[j]  and hash_s2[j] == 0):
                print("hiperonimos",s2[j],s1[i],(sinT[i]).intersection(hipH[j]))
                hash_s1[i] += 1; 
                hash_s2[j] += 1; 
                match += 1; 
                break
            elif ((s2[j] in HipT[i] or len((sinH[j]).intersection(HipT[i]))>0) and s1[i]!=s2[j] and hash_s2[j] == 0):
                print("hiperonimos sobre sinonimos",s2[j],s1[i])
                hash_s1[i] += 1; 
                hash_s2[j] += 1; 
                match += 1; 
                break
            elif len((hipH[j]).intersection(HipT[i]))>0 and s1[i]!=s2[j] and hash_s2[j] == 0: 
                print("hiperonimos3",s2[j],s1[i],(hipH[j]).intersection(HipT[i]))
                hash_s1[i] += 1; 
                hash_s2[j] += 1; 
                match += 1; 
                break
    print(hash_s1)
    print(hash_s2)
    print(match)
    # If there is no match 
    if (match == 0) :
        return 0.0,0.0; 
 
    # Number of transpositions 
    t = 0; 
 
    point = 0; 
 
    # Count number of occurrences 
    # where two characters match but 
    # there is a third matched character 
    # in between the indices 
    for i in range(len1) : 
        if (hash_s1[i]) :
 
            # Find the next matched character 
            # in second string 
            while (hash_s2[point] == 0) :
                point += 1; 
 
            if (s1[i] != s2[point]) :
                point += 1
                t += 1
            else :
                point += 1
                 
    t /= 2; 
    print(t)
    #Return the Jaro Similarity 
    return (( match / len2  +
            (match - t) / match ) / 2.0),t; 

In [7]:
def jaro_distance_contra(s1, s2) :
    antT=[]
    antH=[]
    HipT=[]
    HipH=[]
    sinT=[]
    sinH=[]
    for t in s1:
        #antT.append(bag_of_antonyms(t))
        sinT.append(bag_of_synonyms(t))
        #HipT.append(bag_of_hyperonyms(t))
    for h in s2:
        #antH.append(bag_of_antonyms(h))
        sinH.append(bag_of_synonyms(h))
        #HipH.append(bag_of_hyperonyms(h))
    
    for i in range(len(sinT)):
        tempSet=set()
        tempSett=set()
        for s_ in sinT[i]:
            tempSet=tempSet.union(bag_of_antonyms(s_))
            tempSett=tempSett.union(bag_of_hyperonyms(s_))
        antT.append(tempSet)
        HipT.append(tempSett)
    for i in range(len(sinH)):
        tempSet=set()
        tempSett=set()
        for s_ in sinH[i]:
            tempSet=tempSet.union(bag_of_antonyms(s_))
            tempSett=tempSett.union(bag_of_hyperonyms(s_))
        antH.append(tempSet)
        HipH.append(tempSett)
        
    print("sinonimos de T",sinT)
    print("sinonimos de h",sinH)
    print("antonimos de T",antT)
    print("antonimos de h",antH)
    print("Hiperonimos de T",HipT)
    print("Hiperonimos de H",HipH)

    bandera=True

    # Length of two strings
    len1 = len(s1)
    len2 = len(s2)

    # If the listas de tokens are equal 
    if len1==len2:
        for i in range(len1):
            if s1[i]!=s2[i]:
                bandera=False
                break
        if (bandera):
            return 1.0,0.0; 
 
    if (len1 == 0 or len2 == 0) :
        return 0.0,0.0; 
 
    # Maximum distance upto which matching 
    # is allowed 
    max_dist = (max(len(s1), len(s2)) // 2 ) -1 ; 
 
    # Count of matches 
    match = 0; 
 
    # Hash for matches 
    hash_s1 = [0] * len(s1)
    hash_s2 = [0] * len(s2)
 
    # Traverse through the first string 
    for i in range(len1) : 
            
        # Check if there is any matches 
        for j in range( max(0, i - max_dist),
                    min(len2, i + max_dist + 1)) : 
            #print(s1[i],s2[j])
            # If there is a match or is contain in a bag of sinomys of tk
            if ((s1[i] in antH[j] or s2[j] in antT[i]) and hash_s2[j] == 0) : 
                print(s1[i],antH[j],s2[j],antT[i],"antonimos")
                hash_s1[i] += 1; 
                hash_s2[j] += 1; 
                match += 1; 
                break
            elif (len(HipH[j].intersection(HipT[i]))>0 and s1[i]!=s2[j] and hash_s2[j] == 0):
                print(s1[i],s2[j],HipH[j].intersection(HipT[i]),"antonimos")
                # print(HipT[i])
                # print(HipH[j])
                hash_s1[i] += 1; 
                hash_s2[j] += 1; 
                match += 1; 
                break
    print(hash_s1)
    print(hash_s2)
    print(match)
    # If there is no match 
    if (match == 0) :
        return 0.0,0.0; 
 
    # Number of transpositions 
    t = 0; 
 
    point = 0; 
 
    # Count number of occurrences 
    # where two characters match but 
    # there is a third matched character 
    # in between the indices 
    for i in range(len1) : 
        if (hash_s1[i]) :
 
            # Find the next matched character 
            # in second string 
            while (hash_s2[point] == 0) :
                point += 1; 
 
            if (s1[i] != s2[point]) :
                point += 1
                t += 1
            else :
                point += 1
                 
    t /= 2; 
    print(t)
    #Return the Jaro Similarity 
    return (( match / len2 +
            (match - t) / match ) / 2.0),t; 
 

In [8]:
# entailment
texto="brother_in_law Mohammad Oreibi al-Khalifa new chief judge genocide trial Saddam Hussein has been shot dead police sources say"
hipotesis="Mohammad Khalifa's father_in_law Osama bin_laden"

In [9]:
jaro_distance(texto.lower().split(),hipotesis.lower().split())

sinonimos de T [{'brother_in_law'}, {'muhammad', 'mohammed', 'mahound', 'mahomet', 'mohammad'}, {'oreibi'}, {'al-khalifa'}, {'nouveau', 'raw', 'current', 'new', 'young', 'freshly', 'modern', 'unexampled', 'afresh', 'born_again', 'fresh', 'novel', 'recent', 'newborn', 'strange', 'brand_new'}, {'superintendent', 'flagship', 'head_man', 'overseer', 'chieftain', 'foreman', 'headman', 'organizer', 'head', 'decener', 'chief', 'main', 'principal', 'master', 'primary', 'comptroller', 'administrator'}, {'sentence', 'reckon', 'truth', 'awarder', 'justiciary', 'deem', 'deemer', 'condemn', 'evaluate', 'arbitrate', 'conclude', 'pronounce', 'jurist', 'try', 'doom', 'sheriff', 'appreciate', 'decide', 'esteem', 'believe', 'justice', 'umpire', 'judge', 'evaluator', 'pass_judgment', 'consider', 'think', 'estimate', 'decree', 'regard', 'magistrate', 'determine', 'suppose', 'imagine', 'adjudicate'}, {'racial_extermination', 'genocide', 'race_murder', 'genticide'}, {'trial_run', 'tryout', 'ternary', 'tribu

(0.45, 1.0)

In [10]:
jaro_distance_contra(texto.lower().split(),hipotesis.lower().split())

sinonimos de T [{'brother_in_law'}, {'muhammad', 'mohammed', 'mahound', 'mahomet', 'mohammad'}, {'oreibi'}, {'al-khalifa'}, {'nouveau', 'raw', 'current', 'new', 'young', 'freshly', 'modern', 'unexampled', 'afresh', 'born_again', 'fresh', 'novel', 'recent', 'newborn', 'strange', 'brand_new'}, {'superintendent', 'flagship', 'head_man', 'overseer', 'chieftain', 'foreman', 'headman', 'organizer', 'head', 'decener', 'chief', 'main', 'principal', 'master', 'primary', 'comptroller', 'administrator'}, {'sentence', 'reckon', 'truth', 'awarder', 'justiciary', 'deem', 'deemer', 'condemn', 'evaluate', 'arbitrate', 'conclude', 'pronounce', 'jurist', 'try', 'doom', 'sheriff', 'appreciate', 'decide', 'esteem', 'believe', 'justice', 'umpire', 'judge', 'evaluator', 'pass_judgment', 'consider', 'think', 'estimate', 'decree', 'regard', 'magistrate', 'determine', 'suppose', 'imagine', 'adjudicate'}, {'racial_extermination', 'genocide', 'race_murder', 'genticide'}, {'trial_run', 'tryout', 'ternary', 'tribu

(0.55, 1.5)

In [11]:
jaro_distance_relacionadas(texto.lower().split(),hipotesis.lower().split())

sinonimos de T [{'brother_in_law'}, {'muhammad', 'mohammed', 'mahound', 'mahomet', 'mohammad'}, {'oreibi'}, {'al-khalifa'}, {'nouveau', 'raw', 'current', 'new', 'young', 'freshly', 'modern', 'unexampled', 'afresh', 'born_again', 'fresh', 'novel', 'recent', 'newborn', 'strange', 'brand_new'}, {'superintendent', 'flagship', 'head_man', 'overseer', 'chieftain', 'foreman', 'headman', 'organizer', 'head', 'decener', 'chief', 'main', 'principal', 'master', 'primary', 'comptroller', 'administrator'}, {'sentence', 'reckon', 'truth', 'awarder', 'justiciary', 'deem', 'deemer', 'condemn', 'evaluate', 'arbitrate', 'conclude', 'pronounce', 'jurist', 'try', 'doom', 'sheriff', 'appreciate', 'decide', 'esteem', 'believe', 'justice', 'umpire', 'judge', 'evaluator', 'pass_judgment', 'consider', 'think', 'estimate', 'decree', 'regard', 'magistrate', 'determine', 'suppose', 'imagine', 'adjudicate'}, {'racial_extermination', 'genocide', 'race_murder', 'genticide'}, {'trial_run', 'tryout', 'ternary', 'tribu

(0.35, 0.5)

In [86]:
# Python3 implementation of above approach 
from math import floor
 
# Function to calculate the 
# Jaro Similarity of two strings 
def jaro_distanceO(s1, s2) :
 
    # If the strings are equal 
    if (s1 == s2) :
        return 1.0; 
 
    # Length of two strings 
    len1 = len(s1);
    len2 = len(s2); 
 
    if (len1 == 0 or len2 == 0) :
        return 0.0; 
 
    # Maximum distance upto which matching 
    # is allowed 
    max_dist = (max(len(s1), len(s2)) // 2 ) - 1; 
 
    # Count of matches 
    match = 0; 
 
    # Hash for matches 
    hash_s1 = [0] * len(s1) ;
    hash_s2 = [0] * len(s2) ; 
 
    # Traverse through the first string 
    for i in range(len1) : 
 
        # Check if there is any matches 
        for j in range( max(0, i - max_dist), 
                    min(len2, i + max_dist + 1)) : 
             
            # If there is a match 
            if (s1[i] == s2[j] and hash_s2[j] == 0) : 
                hash_s1[i] = 1; 
                hash_s2[j] = 1; 
                match += 1; 
                break; 
         
    # If there is no match 
    if (match == 0) :
        return 0.0; 
    print(hash_s1)
    print(hash_s2)
    # Number of transpositions 
    t = 0; 
 
    point = 0; 
 
    # Count number of occurrences 
    # where two characters match but 
    # there is a third matched character 
    # in between the indices 
    for i in range(len1) : 
        if (hash_s1[i]) :
 
            # Find the next matched character 
            # in second string 
            while (hash_s2[point] == 0) :
                point += 1; 
 
            if (s1[i] != s2[point]) :
                point += 1;
                t += 1;
            else :
                point += 1;
                 
        t /= 2; 
    print(match)
    print(t)
    # Return the Jaro Similarity 
    return ((match / len2 +
            (match - t) / match ) / 2.0); 
 
# Jaro Winkler Similarity 
def jaro_WinklerO(s1, s2) : 
 
    jaro_dist = jaro_distanceO(s1, s2); 
 
    # If the jaro Similarity is above a threshold 
    if (jaro_dist > 0.7) :
 
        # Find the length of common prefix 
        prefix = 0; 
 
        for i in range(min(len(s1), len(s2))) :
         
            # If the characters match 
            if (s1[i] == s2[i]) :
                prefix += 1; 
 
            # Else break 
            else :
                break; 
 
        # Maximum of 4 characters are allowed in prefix 
        prefix = min(4, prefix); 
 
        # Calculate jaro winkler Similarity 
        jaro_dist += 0.1 * prefix * (1 - jaro_dist); 
 
    return jaro_dist; 
 

In [87]:
jaro_distanceO(texto.split(),hipotesis.split())

[0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1]
7
8.568167686462402e-08


0.9999999938798803

In [12]:
# Python3 implementation of above approach
from math import floor, ceil
 
# Function to calculate the
# Jaro Similarity of two s
def jaro_distance(s1, s2):
     
    # If the s are equal
    if (s1 == s2):
        return 1.0
 
    # Length of two s
    len1 = len(s1)
    len2 = len(s2)
 
    # Maximum distance upto which matching
    # is allowed
    max_dist = floor(max(len1, len2) / 2) - 1
 
    # Count of matches
    match = 0
 
    # Hash for matches
    hash_s1 = [0] * len(s1)
    hash_s2 = [0] * len(s2)
 
    # Traverse through the first
    for i in range(len1):
 
        # Check if there is any matches
        for j in range(max(0, i - max_dist), 
                       min(len2, i + max_dist + 1)):
             
            # If there is a match
            if (s1[i] == s2[j] and hash_s2[j] == 0):
                hash_s1[i] = 1
                hash_s2[j] = 1
                match += 1
                break
 
    # If there is no match
    if (match == 0):
        return 0.0
 
    # Number of transpositions
    t = 0
    point = 0
 
    # Count number of occurrences
    # where two characters match but
    # there is a third matched character
    # in between the indices
    for i in range(len1):
        if (hash_s1[i]):
 
            # Find the next matched character
            # in second
            while (hash_s2[point] == 0):
                point += 1
 
            if (s1[i] != s2[point]):
                t += 1
            point += 1
    t = t//2
 
    # Return the Jaro Similarity
    return (match/ len1 + match / len2 +
            (match - t) / match)/ 3.0
 
# Driver code
s1 = "CRATE"
s2 = "TRACE"
 
# Prjaro Similarity of two s
print(round(jaro_distance(s1, s2),6))
 
# This code is contributed by mohit kumar 29

0.733333


In [12]:
import spacy

# Load English tokenizer, tagger, parser and NER
nlp = spacy.load("en_core_web_md")

# Process whole documents
text = ("When Sebastian Thrun started working on self-driving cars at "
        "Google in 2007, few people get down of the company not took him "
        "seriously. “I can tell you very senior CEOs of major American "
        "car companies would shake my hand and turn away because I wasn’t "
        "worth talking to,” said Thrun, in an interview with Recode earlier "
        "this week.")
doc = nlp(text)

# Analyze syntax
print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])

# Find named entities, phrases and concepts
for entity in doc.ents:
    print(entity.text, entity.label_)

/home/david/anaconda3/envs/rit/lib/python3.9/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11060). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


Noun phrases: ['Sebastian Thrun', 'self-driving cars', 'Google', 'few people', 'the company', 'him', 'I', 'you', 'very senior CEOs', 'major American car companies', 'my hand', 'I', 'Thrun', 'an interview', 'Recode']
Verbs: ['start', 'work', 'drive', 'get', 'take', 'tell', 'shake', 'turn', 'talk', 'say']
Sebastian Thrun PERSON
Google ORG
2007 DATE
American NORP
Thrun FAC
Recode ORG
earlier this week DATE


In [91]:
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

When when SCONJ WRB advmod Xxxx True True
Sebastian Sebastian PROPN NNP compound Xxxxx True False
Thrun Thrun PROPN NNP nsubj Xxxxx True False
started start VERB VBD advcl xxxx True False
working work VERB VBG xcomp xxxx True False
on on ADP IN prep xx True True
self self NOUN NN npadvmod xxxx True False
- - PUNCT HYPH punct - False False
driving drive VERB VBG amod xxxx True False
cars car NOUN NNS pobj xxxx True False
at at ADP IN prep xx True True
Google Google PROPN NNP pobj Xxxxx True False
in in ADP IN prep xx True True
2007 2007 NUM CD pobj dddd False False
, , PUNCT , punct , False False
few few ADJ JJ amod xxx True True
people people NOUN NNS nsubj xxxx True False
get_down get_down NOUN NN appos xxx_xxxx False False
of of ADP IN prep xx True True
the the DET DT det xxx True True
company company NOUN NN pobj xxxx True False
not not PART RB neg xxx True True
took take VERB VBD ROOT xxxx True False
him he PRON PRP dobj xxx True True
seriously seriously ADV RB advmod xxxx True Fal

In [35]:
def jaro_distance_prueba(s1, s2) :
    sinT=[]
    sinH=[]
    HipT=[]
    hipH=[]
    HipH=[]
    antT=[]
    antH=[]
    for t in s1:
        sinT.append(bag_of_synonyms(t))
    for h in s2:
        sinH.append(bag_of_synonyms(h))
    
    tempSet=set()
    tempSet1=set()
    for i in range(len(sinT)):
        tempSet=set()
        tempSet1=set()
        for s_ in sinT[i]:
            tempSet=tempSet.union(bag_of_hyperonyms(s_))
            tempSet1=tempSet1.union(bag_of_antonyms(s_))
        antT.append(tempSet1)
        HipT.append(tempSet)
    for i in range(len(sinH)):
        tempSet=set()
        tempSet1=set()
        tempSett2=set()
        for s_ in sinH[i]:
            tempSet=tempSet.union(bag_of_hyponyms(s_))
            tempSet1=tempSet1.union(bag_of_antonyms(s_))
            tempSett2=tempSett2.union(bag_of_hyperonyms(s_))
        antH.append(tempSet1)
        HipH.append(tempSett2)
        hipH.append(tempSet)
            
        
    print("sinonimos de T",sinT)
    print("Hiperonimos de T",HipT)
    print("sinonimos de H",sinH)
    print("hiponimos de h",hipH)

    bandera=True

    # Length of two strings
    len1 = len(s1)
    len2 = len(s2)

    # If the listas de tokens are equal 
    if len1==len2:
        for i in range(len1):
            if s1[i]!=s2[i]:
                bandera=False
                break
        if (bandera):
            return 1.0,0.0; 
 
    if (len1 == 0 or len2 == 0) :
        return 0.0,0.0; 
 
    # Maximum distance upto which matching 
    # is allowed 
    max_dist = (max(len(s1), len(s2)) // 2 )-1 ; 
 
    # Count of matches 
    match = 0; 
    match1 = 0; 
 
    # Hash for matches 
    hash_s1 = [0] * len(s1)
    hash_s2 = [0] * len(s2)
 
    # Traverse through the first string 
    for i in range(len1) : 
            
        # Check if there is any matches 
        for j in range( max(0, i - max_dist),
                    min(len2, i + max_dist + 1)) : 
            print(s1[i],s2[j])
            # If there is a match or is contain in a bag of sinomys of tk
            if ((s1[i] == s2[j] or s1[i] in sinH[j] or s2[j] in sinT[i]) and hash_s2[j] == 0) : 
                print(s1[i],s2[j],"sinonimos")
                hash_s1[i] += 1; 
                hash_s2[j] += 1; 
                match += 1; 
                break
            elif ((s1[i] in hipH[j] or len((sinT[i]).intersection(hipH[j]))>0) and hash_s2[j] == 0):
                print("hiperonimos",s2[j],s1[i],(sinT[i]).intersection(hipH[j]))
                hash_s1[i] += 1; 
                hash_s2[j] += 1; 
                match += 1; 
                break
            elif ((s2[j] in HipT[i] or len((sinH[j]).intersection(HipT[i]))>0) and hash_s2[j] == 0): 
                print("hiperonimos sobre sinonimos",s2[j],s1[i])
                hash_s1[i] += 1; 
                hash_s2[j] += 1; 
                match += 1; 
                break
            elif len((hipH[j]).intersection(HipT[i]))>0 and hash_s2[j] == 0: 
                print("hiperonimos3",s2[j],s1[i],(hipH[j]).intersection(HipT[i]))
                hash_s1[i] += 1; 
                hash_s2[j] += 1; 
                match += 1; 
                break
            elif ((s1[i] in antH[j] or s2[j] in antT[i]) and hash_s2[j] == 0) : 
                print(s1[i],antH[j],s2[j],antT[i],"antonimos")
                hash_s1[i] = 2; 
                hash_s2[j] = 2; 
                match1 += 1; 
                break
            elif (len(HipH[j].intersection(HipT[i]))>0 and s1[i]!=s2[j] and hash_s2[j] == 0):
                print(s1[i],s2[j],HipH[j].intersection(HipT[i]),"antonimos")
                # print(HipT[i])
                # print(HipH[j])
                hash_s1[i] = 2; 
                hash_s2[j] = 2; 
                match1 += 1; 
                break
    print(hash_s1)
    print(hash_s2)
    print("inferencias",match)
    print("contradicciones",match1)
    # If there is no match 
    if (match == 0) :
        return 0.0,0.0; 
 
    # Number of transpositions 
    t = 0; 
 
    point = 0; 
 
    # Count number of occurrences 
    # where two characters match but 
    # there is a third matched character 
    # in between the indices 
    for i in range(len1) : 
        if (hash_s1[i]) :
 
            # Find the next matched character 
            # in second string 
            while (hash_s2[point] == 0) :
                point += 1; 
 
            if (s1[i] != s2[point]) :
                point += 1
                t += 1
            else :
                point += 1
                 
    t /= 2; 
    print(t)
    #Return the Jaro Similarity 
    return ((match / len2) - (match1 /len2) +
            (match - t) / match ) / 3.0; 

In [38]:
texto="Swansea striker Lee Trundle has negotiated a lucrative image-rights deal with the League One club"
hipotesis="Lee Trundle is in business with the League One club"

In [39]:
jaro_distance_prueba(texto.lower().split(),hipotesis.lower().split())

sinonimos de T [{'swansea'}, {'hitter', 'striker', 'attacker'}, {'robert_edward_lee', 'tsung_dao_lee', 'shelton_jackson_lee', 'downwind', 'richard_henry_lee', 'rose_louise_hovick', 'lee_yuen_kam', 'lee_side', 'lighthorse_harry_lee', 'lee', 'leeward'}, {'trundle_bed', 'lantern_wheel', 'trundle'}, {'hath', 'hasn’t', 'has', 'haves'}, {'negotiated'}, {'o', 'rix', 'å', 'aud', 'an', 'deoxyadenosine_monophosphate', 'ace', 'au', 'group', 'la_dièse', 'a', 'ette', 'ess', 'ampere', 'vitamin', 'australian_dollar', 'type', 'adenine', 'euse', 'angstrom', 'arc', 'plus', 'she'}, {'profitable', 'moneymaking', 'lucrative', 'economic', 'remunerative'}, {'image-rights'}, {'steal', 'bargain', 'administer', 'treatment', 'apportion', 'hand', 'conduct', 'handle', 'sell', 'pitch', 'cope', 'manage', 'distribute', 'deal', 'consider', 'batch', 'softwood', 'contract', 'trade', 'share', 'cover', 'agreement', 'allotment', 'business_deal'}, {'in_case_of', 'across_from', 'w', 'with', 'opposed_to', 'gas', 'c̄', 'in_exc

0.2857142857142857